In [9]:
# ESTUDO DO PROBLEMA DE ESCALONAMENTO DE HORÁRIOS E ALGORITMOS GENÉTICOS
# {Inst,Salas,Profs,Programas,Cursos,Semestres} -> {Horario Semanal para cada programa com n semestres}

# Cada programa terá vários cursos
# Cada programa terá n semestres
# Cada curso pertencerá à um semestre especifico
# Cada semestre não poderá superar mais do que 10 aulas semanais no horario

# AG -> Inst{ Programas{ Cursos, Semestres }, Salas, Profs{Cursos} } : Schedule

# *OBS1: Haverá "ataques ou choque de horarios" em um schedule para programa com n semestres (ataque entre semestres)
# *OBS2: Após a geração de um schedule pra um programa, guardar restrições desse programa para aplicar em outro, caso use os mesmos requisitos (salas e professores)


# ------------ AG ------------
# ___Code___
# Laço:
# - Popular
# - Fitness
# - Testar -> (Sol, S/ Sol, Evoluir)
# Fim

# Evoluir:
# - Seleção
# - Cruzamento
# - Mutação
# __________

# Conceitos:
# • Definir o que é cromossomo
# • Teste de Meta: Sim, se algum cromossomo tem fitness = 1. Não, caso o contrário.
# • Fitness: (1 - Erro/MaxErro)
#                  (Acerto/MaxAcerto)
# • Seleção: Roleta Viciada (Gera 2*n casais de cromossomos. Partição da roleta é baseada no valor Fitness)
# • Cruzamento: Cruzar casais gerados na Seleção. Ao cruzar, é pegada informação do macho e da fêmea, e são gerados 2 filhos melhores geneticamente.
# • Mutação: Mutar gene de cromossomos aleatoriamente

In [10]:
#Import .ipynb as module: %run name.ipynb
import math
import random
import copy
import itertools as it

In [11]:
class Schedule:
    #     10 Aulas semanais:
    # | Seg  Ter  Qua  Qui  Sex |
    # |  1    3    5    7    9  |
    # |  2    4    6    8   10  |
    #
    def __init__(self):
        self.weekName = ['Seg', 'Ter', 'Qua', 'Qui', 'Sex']
        self.week = [[0]*2]*5
    
    def __repr__(self):
        strOut = '| '
        for i in self.weekName:
            strOut += i + '  '
        strOut += ' |\n| '
        for i in self.week:
            strOut += ' ' + str(i[0]) + '   '
        strOut += ' |\n| '
        for i in self.week:
            strOut += ' ' + str(i[1]) + '   '
        strOut += ' |\n '
        return strOut
    
Schedule()

| Seg  Ter  Qua  Qui  Sex   |
|  0    0    0    0    0    |
|  0    0    0    0    0    |
 

In [12]:
class Curso:
    def __init__(self):
        return
        
class Semestre:
    def __init__(self):
        return

class Programa:
    def __init__(self):
        return
        
class Sala:
    def __init__(self):
        return
    
class Professor:
    def __init__(self):
        return